In [1]:
# Keep Colab alive

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [2]:
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18913 sha256=958a58833d33da9caff25c82f56b46e9c15c885a029f46a971a13ba0d487f21d
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer 
from keras.utils import pad_sequences
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.utils import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%ls /content/drive/MyDrive/Datasets/

Mounted at /content/drive
xsum/  xsum_reduced_prepared_150_50.csv


In [5]:
# sequences config:
max_text_len=150
max_summary_len=50

In [6]:
# load
prepared = pd.read_csv('/content/drive/MyDrive/Datasets/xsum_reduced_prepared_150_50.csv')
prepared

,Unnamed: 0,text,summary
0,0,the national oceanic and atmospheric administration noaa forecasts 14 to 23 named storms eight to 14 of these could develop into hurricanes it says.the season could be one of the most active on re...,sostok _START_ as many as 14 hurricanes could hit the atlantic basin this year the top us climate agency says. _END_ eostok
1,1,it was rare example of the cold war turning hot pitting the us and its allies against the ussr north korea and communist china it was marked by dramatic swings of fortune and devastating death tol...,sostok _START_ in 1950 as the international community was coming to terms with the aftermath of world war ii new conflict broke out at the edge of the asian continent. _END_ eostok
2,2,the singer and factor judge cited unreasonable behaviour in the action filed at the high court in london.the high profile couple split in february after claims the england star had been unfaithful...,sostok _START_ cheryl cole has filed for divorce from footballer husband ashley cole it has been confirmed. _END_ eostok
3,3,ofcom said they had received handful of complaints about the act which saw mr starr swallow lightbulb among other items.some viewers were concerned that it would encourage children to try dangerou...,sostok _START_ complaints about britain got talent are being looked at by the media watchdog after viewers raised concerns about glass swallower stevie starr. _END_ eostok
4,4,napper was linked to series of rapes and attempted rapes in south london parkland known as the green chain but was cleared for being too tall .he went on to kill rachel nickell in 1992 and samanth...,sostok _START_ serial rapist robert napper was brought to the attention of metropolitan police as early as 1989 but series of bad errors allowed the 42-year-old to slip the net report by the indep...
...,...,...,...
47995,47995,the new zealand racer set the fastest time of the week so far on his superbike 124.888mph before achieving the fastest ever lightweight lap.his 118.812mph lightweight lap on 250cc honda was well w...,sostok _START_ bruce anstey again set the pace in the third qualifying session for the 2017 festival of motorcycling on wednesday. _END_ eostok
47996,47996,city paid their premier league counterparts £25m for the ivory coast forward two and half years ago.no fee has been agreed as yet and talks may continue into next week but it is anticipated bony w...,sostok _START_ swansea city are close to securing the return of striker wilfried bony from manchester city. _END_ eostok
47997,47997,media playback is not supported on this devicekeetels member of the victorious 2014 world cup team fired low past maddie hinch in the fourth quarter after two shots had been blocked.england won se...,sostok _START_ england hopes of defending their eurohockey title were thwarted at the semi-final stage as marloes keetels strike sent hosts netherlands through. _END_ eostok
47998,47998,the chorus of taylor new single look what you made me do follows the same rhythmic pattern as too sexy.after the song was released overnight the duo tweeted thank you taylor swift what marvellous ...,sostok _START_ right said fred have been credited as writers on taylor swift new single in one of the most bizarre musical combinations of the year. _END_ eostok


In [7]:
# data more prepare
x_tr,x_val,y_tr,y_val=train_test_split(np.array(prepared['text']),np.array(prepared['summary']),test_size=0.1,random_state=0,shuffle=True)

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

# rare word analysis - X
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data - X
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

#prepare a tokenizer for reviews on training data - Y
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

# rare word analysis - Y
thresh=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

# remove empty summaries
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

print("Size of vocabulary from the w2v model = {}".format(x_voc))


% of rare words in vocabulary: 63.383132872431105
Total Coverage of rare words: 1.9008429752383218
Size of vocabulary in X = 30932
% of rare words in vocabulary: 73.60875073386275
Total Coverage of rare words: 4.367686488532698
Size of vocabulary in Y = 8542
Size of vocabulary from the w2v model = 30932


In [8]:
# model setup

K.clear_session()

latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print(model.summary())
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 150, 200)     6186400     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 150, 300),   601200      ['embedding[0][0]']              
                                 (None, 300),                                                     
                                 (None, 300)]                                                     
                                                                                              

In [9]:
# train setup
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

checkpoint_filepath = '/content/drive/MyDrive/checkpoint.best.hdf'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

try:
  model.load_weights(checkpoint_filepath)
  print('Loaded model checkpoint.')
except:
  print('Cannot load weights from checkpoint, it may not exist yet.')

Loaded model checkpoint.


In [12]:
# training
do_train = False

if do_train:
  history = model.fit(
      [x_tr, y_tr[:, :-1]],
      y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:],
      epochs=50,
      callbacks=[es, model_checkpoint_callback],
      batch_size=256,
      validation_data=(
          [x_val, y_val[:, :-1]],
          y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:, 1:],
      ),
  )
  # training visualisation
  from matplotlib import pyplot
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.legend()
  pyplot.show()

In [15]:
# target to source dictionary setup and decoder

reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c], verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

# finally apply model
for i in range(0,10):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: the collision happened between junction 10 for winchester at bar end and junction at winnall just after 10 20 bst initially all three lanes of the northbound carriageway were closed by police for the rest of the afternoon although one lane has since reopened one person was taken to hospital with serious injuries following the crash motorists were told to avoid the area and delays of up to two hours were reported with traffic back to junction 13 for eastleigh surrounding roads including in and out of winchester and through have also faced disruption the hampshire and isle of wight air ambulance was called to the scene a spokesman for south central ambulance service said one patient with serious injuries was taken to southampton general hospital one patient with minor injuries to the royal hampshire county hospital winchester and third patient with minor injuries was treated at the scene 
Original summary: start a serious crash involving three lorries and car caused the closure o

In [31]:
sekv = x_tokenizer.texts_to_sequences(['sostok _START_ cheryl cole has filed for divorce from footballer husband ashley cole it has been confirmed. _END_ eostok'])
padded = pad_sequences(sekv, maxlen=max_summary_len, padding='post')
decode_sequence(padded)

1/1 [==============================] - 0s 71ms/step


' start britain won the first round of the world cup with victory over the netherlands end'

??? why